In [26]:
import requests
import gradio as gr
from PIL import Image
from io import BytesIO

def request_gpt(user_text, temperature=0.7, top_p=0.95, max_tokens=800):

    # endpoint
    # method
    # header
    # payload

    api_base="https://team10-eighti.openai.azure.com" 
    deployment_id="gpt-4o"  
    endpoint = f"{api_base}/openai/deployments/{deployment_id}/chat/completions?api-version=2024-08-01-preview"
    api_key = '1BajXTI5Mp0tKrs46XFGuOWjSPXKzOZSKy8e6R3qha1SVQ4lz1PFJQQJ99ALACYeBjFXJ3w3AAABACOGgPvw'
    
    search_endpoint="https://team10-eighti-search.search.windows.net"
    search_key = 'wnalAsW6FqKRHIR6S3sUZGzNH28Lf3sBOS2ubCZsZxAzSeA205k3'
    search_index="sign-index"
    semantic_name = "sign-semantic"

    method = requests.post
    headers = {
        "Content-Type": "application.json",
        "api-key": api_key
    }
    payload = {
        "messages": [
            {
                "role": "system",
                "content": "당신은 사용자가 수화 정보를 찾는데 도움을 주는 수화 AI 도우미입니다."
            },
            {
                "role": "user",
                "content": user_text
            }
        ],
        "temperature": temperature,
        "top_p": top_p,
        "frequency_penalty": 0,
        "presence_penalty": 0,
        "max_tokens": max_tokens,
        "stop": None,
        "data_sources": [
            {
                "type": "azure_search",
                "parameters": {
                    "endpoint": search_endpoint,
                    "index_name": search_index,
                    "semantic_configuration": semantic_name,
                    "query_type": "semantic",
                    "fields_mapping": {},
                    "filter": None,
                    "top_n_documents": 5,
                    "authentication": {
                        "type": "api_key",
                        "key": search_key
                    }
                }
            }
        ]
    }

    response = requests.post(endpoint, headers=headers, json=payload)
    # print(response.status_code)
    # print(response.text)

    if response.status_code == 200:
        response_json = response.json()
        content_text = response_json['choices'][0]['message']['content']
        # citations = response_json['choices'][0]['message']['context']['citations'][0]['content'].split('\n')
        citations = response_json['choices'][0]['message']['context']['citations']
        citations_movie = get_citations(citations, "mp4")
        citations_images = get_citations(citations, "jpg").split(' ')
        print("-----------")
        print(citations_images)
        citations_images_first = None
        citations_images_second = None

        if (len(citations_images) > 0 ):
            citations_images_first = get_image_url(citations_images[0])

            print(citations_images_first)
            if (len(citations_images) > 1 ):
                citations_images_second = citations_images[1]

        return content_text, citations_movie, citations_images_first, citations_images_second
    else:
        return ""

def get_citations(citations="", extentions="mp4"):
    print(len(citations))
    if len(citations) > 0 and len(citations[0]) > 0 and len(citations[0]['content']) > 0:
        contents = citations[0]['content'].split('\n')

        for row in contents:
            row_list = row.split('.')
            if row_list[-1] == extentions:
                print("extentions=", extentions, "|row=",  row)
                return row

    return ""

def get_image_url(url):

    if not url.strip():
        return None  # 빈값 처리
    try:
        response = requests.get(url)
        response.raise_for_status()  # HTTP 에러 확인
        img = Image.open(BytesIO(response.content))  # 이미지를 메모리에서 읽음
        return img
    except Exception as e:
        return f"Error fetching image: {str(e)}"

def click_send(prompt, histories):
    print(prompt)
    print(histories)
    # history_list = get_history_messages(histories=histories)
    # response_text, citation_html = request_gpt(prompt, history_list)
    response_text, citations_movie, citations_images_first, citations_images_second = request_gpt(prompt)
    histories.append((prompt, response_text))
    return histories, "", citations_movie, citations_images_first, citations_images_second

with gr.Blocks() as demo:

    with gr.Row():
        with gr.Column():
            chatbot = gr.Chatbot(label='채팅 기록')
            with gr.Row():
                input_textbox = gr.Textbox(label="", scale=7)
                send_button = gr.Button("전송", scale=1)

        # citation = gr.HTML(label='참조')
        with gr.Column():
            videio = gr.Video(label="Video Player", autoplay=True)
            with gr.Row():
                image_first = gr.Image(scale=0.3)
                image_second = gr.Image(scale=0.3)


    input_textbox.submit(fn=click_send, inputs=[input_textbox, chatbot], outputs=[chatbot, input_textbox, videio, image_first, image_second])
    send_button.click(fn=click_send, inputs=[input_textbox, chatbot], outputs=[chatbot, input_textbox, videio, image_first, image_second])

# content_text, citations_list = request_gpt("된장찌개가 수화로 어떻게 돼?", temperature=0.7, top_p=0.95, max_tokens=800)

# print(content_text)
# print(citations_list)

demo.launch()


c:\Users\USER\AppData\Local\Programs\Python\Python312\Lib\site-packages\gradio\components\chatbot.py:237: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  warnings.warn(
c:\Users\USER\AppData\Local\Programs\Python\Python312\Lib\site-packages\gradio\components\base.py:201: UserWarning: 'scale' value should be an integer. Using 0.3 will cause issues.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7883

To create a public link, set `share=True` in `launch()`.


된장찌개
[]
1
extentions= mp4 |row= http://sldict.korean.go.kr/multimedia/multimedia_files/convert/20201210/784487/MOV000356911_700X466.mp4
1
extentions= jpg |row= http://sldict.korean.go.kr/multimedia/multimedia_files/convert/20201224/791296/IMG000357950_700X466.jpg http://sldict.korean.go.kr/multimedia/multimedia_files/convert/20151126/217119/IMG000208379_700X466.jpg
-----------
['http://sldict.korean.go.kr/multimedia/multimedia_files/convert/20201224/791296/IMG000357950_700X466.jpg', 'http://sldict.korean.go.kr/multimedia/multimedia_files/convert/20151126/217119/IMG000208379_700X466.jpg']
<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=341x466 at 0x16F8DF1BAD0>
